In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import figure
import seaborn as sns

In [ ]:
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_csv('../input/bike-sharing-demand/train.csv', parse_dates = ['datetime'])
test=pd.read_csv('../input/bike-sharing-demand/test.csv', parse_dates = ['datetime'])
sampsub = pd.read_csv("../input/bike-sharing-demand/sampleSubmission.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
figure(figsize=(12, 8))
sns.heatmap(train.corr())

In [ ]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['weekday'] = train['datetime'].dt.weekday

In [ ]:
train = train.drop('datetime', axis = 1)

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['weekday'] = test['datetime'].dt.weekday

In [ ]:
test = test.drop('datetime', axis = 1)

In [ ]:
col = ['temp', 'atemp', 'humidity', 'windspeed']

In [ ]:
for i in col:
    scaler = StandardScaler()
    train[col]=scaler.fit_transform(train[col])
    test[col]=scaler.transform(test[col])   

In [ ]:
train['count'].hist()

In [ ]:
X_train = train.drop(['count', 'casual','registered'], axis = 1)
y_train = train['count']

In [ ]:
y_train = np.log(y_train.values)

In [ ]:
X_train.shape, y_train.shape, test.shape

In [ ]:
lr_range = [0.01, 0.1, 0.2,0.25]
n_est_range = [30,50,100]
md_range = [4,5,6,7,8]

In [ ]:
parameters = dict(learning_rate=lr_range, n_estimators=n_est_range, max_depth=md_range)

In [ ]:
clf = XGBRegressor(random_state=1211)

In [ ]:
kfold = KFold(n_splits=5, random_state=42,shuffle=True)

In [ ]:
cv = GridSearchCV(clf, param_grid=parameters, cv=kfold,scoring='r2')

In [ ]:
cv.fit(X_train,y_train)

In [ ]:
y_pred = cv.predict(test)

In [ ]:
y_pred = np.exp(y_pred)

In [ ]:
y_pred = y_pred.round()

In [ ]:
submission = pd.DataFrame({ "datetime": sampsub.datetime, "count": y_pred})

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission_XGBoost_Grid_Search.csv", index=False)